# ChatGPT Investigates Twitter's Tweet Sentiment


## Introduction :-
### Welcome to this light-hearted notebook where we will embark on a detective journey with ChatGPT to investigate the sentiment of tweets from the infamous world of Twitter. With a touch of humor, we'll explore how ChatGPT can assist us in deciphering the emotional undertones of tweets.

## Case Brief:
### Our mission is to analyze the sentiment of tweets and determine whether they are positive, negative, or neutral and also find out the specific context that will make that tweet positive, negative or neutral. ChatGPT will serve as our trusty sidekick, helping us make sense of the vast sea of tweets and uncover any hidden emotions lurking in the Twitterverse.



### Importing ChatGPT Libraries

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['open_ai_key']
# print(openai.api_key)

### Importing Basic Libraries

In [2]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split

/home/aryan/miniconda3/envs/ML/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df_train = pd.read_csv('/home/aryan/Desktop/Extra/train.csv')
df_test = pd.read_csv('/home/aryan/Desktop/Extra/test.csv')
df_submission = pd.read_csv('/home/aryan/Desktop/Extra/sample_submission.csv')
df_sub_73 = pd.read_csv('/home/aryan/Desktop/Extra/submission_73.5.csv')

In [4]:
df_test.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [5]:
df_train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [6]:
df_train.shape

(27481, 4)

### Split the Data into Train and Test

In [7]:
train, test = train_test_split(df_train, stratify=df_train['sentiment'].tolist(), test_size=0.01, random_state=42)

In [8]:
test = test.reset_index(drop=True)

In [9]:
test.head()

,textID,text,selected_text,sentiment
0,33537ffd4d,is excited to see Samantha&Denise again http:...,excited,positive
1,609bf831e2,apparently even Novell Moonlight doesn`t work ...,doesn`t work,negative
2,4db58698d0,__ Aww bless her. She`s one of my faves,bless her. She`s one of my faves,positive
3,fb45b19ca3,hahah I`ll try,hahah I`ll try,neutral
4,630285dbc7,bored,bored,neutral


### Prompt Engineering

In [10]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [11]:
delimiter = "####"
system_message = f"""
consider yourself as Human working on positive , negative or neural text classifier having great
deal of english language. Now given a text delimited with {delimiter} characters.
Highlight the parts of text which make it positive or negative or neutral.
Output should be in python dictionary format with keys being sentiment, part_of_text
and the values for sentiment and part_of_text should be single output string and should not have list of outputs for better understanding see examples bellow
where part_of_text contains only that words that have high similarity with the sentiments giving out different parts
of text which make it profane, vulgar, or offensive
Note that when sentiment is neutral, part_of_text simply contains user message and part_of_text contains only one index size

So, I need to train on these below examples
there will be 3 columns input_text,text_extracted,labels these columns will have values separated by comma 
and their descriptions are as follows
input_text = actual text data
text_extracted = text which actually provides emotions to the original text
labels = labels/sentiment assigned to extracted text

1. you are stupid , stupid ,negative
2. Sooo SAD I will miss you here in San Diego!!!, Sooo SAD, negative
3. my boss is bullying me..., bullying me, negative
4. what interview! leave me alone, leave me alone, negative
5. Sons of bitch why couldn`t they put them on the releases we already bought, Sons of bitch, negative

6. A little happy for the wine jeje ok it`sm my free time so who cares jaja i love this day, A little happy fo, positive
7. lucky kid...i so wanna see loserville  pity im in oz...., lucky, positive
8. Happy Mothers day to all you Mums out there, Happy Mothers day to all you Mums out there, positive
9. Thank you, Afrin Nasal Spray! Also, I got a giant teacup tonight!, Thank you,, positive
10. Packing up for the trip to Ascari track, thank you Bacardi, thank you Bacardi, positive

11. yeah, it`s a bit 80s silly at points but i love it, yeah, it`s a bit 80s silly at points but i love it, neutral
12. It's gonna be a super long day, It`s gonna be a super long day, neutral
13. I'm soooooooo hungry right now. DH is still not home, I`m soooooooo hungry right now. DH is still not home., neutral
14. Where`d the songs go on the site I want 'Do You' on this computer too, Where`d the songs go on the site I want 'Do You' on this computer too, neutral
15. fighting again, fighting again, neutral
"""

In [12]:
user_message = f"""\
 its cool. i ate too much ice cream
"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

{'sentiment': 'positive', 'part_of_text': 'its cool. i ate too much ice cream'}


In [13]:
test['predicted_text'] = None
test['predicted_sentiment'] = None

### Note :-
### In our quest to explore the realm of GPT-powered language generation, we find ourselves relying on the Time library as our trusty companion. With our current limitations in GPT subscription, we embrace the art of patience, gracefully waiting for a minute before unleashing the power of three requests simultaneously. Like a well-choreographed symphony, we synchronize our actions with time, ensuring a harmonious flow in our interactions with GPT.

In [14]:
from time import sleep
for j,i in enumerate(test.index):
    if i%3==0:
        sleep(60)          # To manage rate limits and stay within the maximum number of requests per minute without a subscription
    user_message = test['text'][i]
    print(f'--->{user_message}\n')
    messages =  [  
    {'role':'system', 
     'content': system_message},    
    {'role':'user', 
     'content': f"{delimiter}{user_message}{delimiter}"},  
    ] 
    response = get_completion_from_messages(messages)
    try:
        response = json.loads(response.replace("'", '"'))
        print(j,response)
        print('\n')
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", j,e)
        continue
    selected_text = response.get('part_of_text')
    selected_sentiment = response.get('sentiment')
#     print(selected_text)
    test['predicted_text'][j] = selected_text
    test['predicted_sentiment'][j] = selected_sentiment
#     break

--->is excited to see Samantha&Denise again  http://plurk.com/p/rr5id

0 {'sentiment': 'positive', 'part_of_text': 'excited to see'}


--->apparently even Novell Moonlight doesn`t work on my Jaunty

1 {'sentiment': 'negative', 'part_of_text': 'doesn`t work'}


--->__ Aww bless her. She`s one of my faves

2 {'sentiment': 'positive', 'part_of_text': 'Aww bless her. She`s one of my faves'}


---> hahah I`ll try

3 {'sentiment': 'neutral', 'part_of_text': 'hahah I`ll try'}


---> bored

4 {'sentiment': 'neutral', 'part_of_text': 'bored'}


---> that was such a tease!  I cant wait til Sunday!!  but then after that, nothing until November   oh well, i can wait!

5 {'sentiment': 'neutral', 'part_of_text': 'that was such a tease!  I cant wait til Sunday!!  but then after that, nothing until November   oh well, i can wait!'}


--->  Joe and I loveee you.

6 {'sentiment': 'positive', 'part_of_text': 'loveee'}


--->homeee! my feet are sore

7 {'sentiment': 'neutral', 'part_of_text': 'homeee! my 

48 {'sentiment': 'negative', 'part_of_text': 'Burnt the **** out of my hand today.'}


--->My back is killing me. Stupid softball

49 {'sentiment': 'negative', 'part_of_text': 'My back is killing me. Stupid softball'}


---> yeah yeah gd night then

50 {'sentiment': 'neutral', 'part_of_text': 'yeah yeah gd night then'}


--->#MQM has opted to strike, May 12th - Altaf bhai to unveil what really happened in #Karachi  http://tinyurl.com/c73ehq

51 {'sentiment': 'negative', 'part_of_text': 'opted to strike, May 12th - Altaf bhai to unveil what really happened in #Karachi'}


---> Brilliant  Thanks! Not sure what brownie points will get you though!

52 {'sentiment': 'positive', 'part_of_text': 'Brilliant  Thanks!'}


---> no sleeping for me im at work   it will be summer break soon. Tell them to get there butts out of bed lol

53 {'sentiment': 'neutral', 'part_of_text': 'no sleeping for me im at work   it will be summer break soon. Tell them to get there butts out of bed lol'}


---> have a

98 {'sentiment': 'positive', 'part_of_text': 'thanks thoughtful ladies feel special'}


---> Haha, she`s up here to stay, it`s nearly 2 hours on the coach for her to get here, may not be convenient for an hour of TV!

99 {'sentiment': 'neutral', 'part_of_text': 'Haha, she`s up here to stay, it`s nearly 2 hours on the coach for her to get here, may not be convenient for an hour of TV!'}


---> everyone loves u sarah not just the tweeters! the today show couldnt stop raving about your beautifulness...

100 {'sentiment': 'positive', 'part_of_text': 'loves beautifulness'}


---> Ooo showing of your French skills!! lol Things good over here. Lovely weather, so should be outside  How`s u?

101 {'sentiment': 'positive', 'part_of_text': 'Lovely weather'}


---> Thanks

102 {'sentiment': 'neutral', 'part_of_text': 'Thanks'}


--->I`m torn, do I get an extended warranty for my iPhone when mine runs out, or jailbreak? I`ve already got problems with battery & earpiece

103 {'sentiment': 'neutral',

146 {'sentiment': 'positive', 'part_of_text': 'malibu is the drink for me'}


--->_raven oh, I`m skeptical too. I just think it`s the best thing he could have said under the circumstances

147 {'sentiment': 'neutral', 'part_of_text': 'oh, I`m skeptical too. I just think it`s the best thing he could have said under the circumstances'}


--->Please visit http://vzerohost.com/info and sign up to alpha test a image hosting service!

148 {'sentiment': 'neutral', 'part_of_text': 'Please visit http://vzerohost.com/info and sign up to alpha test a image hosting service!'}


--->wishing my days off were the same days that people wanted to party.

149 {'sentiment': 'neutral', 'part_of_text': 'wishing my days off were the same days that people wanted to party.'}


--->back from lunch and it`s pouring rain! ugh  ohh well at least it gets me into a Death Cab For Cutie mood!

150 {'sentiment': 'neutral', 'part_of_text': 'back from lunch and it`s pouring rain! ugh  ohh well at least it gets me into a

197 {'sentiment': 'negative', 'part_of_text': 'flu earlier this month'}


--->Twitter you suck

198 {'sentiment': 'negative', 'part_of_text': 'you suck'}


---> how sweet

199 {'sentiment': 'positive', 'part_of_text': 'how sweet'}


---> I`m so honored to hear my name on a Breezy track  hope u liked my ****! #drunk_bowling

200 {'sentiment': 'positive', 'part_of_text': 'honored, liked', 'original_text': 'I`m so honored to hear my name on a Breezy track  hope u liked my ****! #drunk_bowling'}


---> Happened to me the other day.  Was wearing a necklace that made a white spot on my chest.

201 {'sentiment': 'neutral', 'part_of_text': 'Happened to me the other day.  Was wearing a necklace that made a white spot on my chest.'}


--->mmm... hot chocolate, ugboots and topgear... life is good.

202 {'sentiment': 'positive', 'part_of_text': 'hot chocolate, life is good'}


---> Your schmoo is moving? Are you sad?

203 {'sentiment': 'neutral', 'part_of_text': 'Your schmoo is moving? Are you sad

RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 5283234cdebbf744d3262237650f4f83 in your message.)

In [17]:
test.head(221)

,textID,text,selected_text,sentiment,predicted_text,predicted_sentiment
0,33537ffd4d,is excited to see Samantha&Denise again http:...,excited,positive,excited to see,positive
1,609bf831e2,apparently even Novell Moonlight doesn`t work ...,doesn`t work,negative,doesn`t work,negative
2,4db58698d0,__ Aww bless her. She`s one of my faves,bless her. She`s one of my faves,positive,Aww bless her. She`s one of my faves,positive
3,fb45b19ca3,hahah I`ll try,hahah I`ll try,neutral,hahah I`ll try,neutral
4,630285dbc7,bored,bored,neutral,bored,neutral
...,...,...,...,...,...,...
216,aea00d01f6,i`m not being mean,i`m not being mean,positive,i`m not being mean,neutral
217,d5c1749c3a,my teeth and head hurts,my teeth and head hurts,negative,teeth and head hurts,negative
218,94b2a982ef,umm..nope not anymoree,umm..nope not anymoree,neutral,umm..nope not anymoree,neutral
219,72dffbd0ef,"I will, n btw happy mothers day sissy","I will, n btw happy mothers day sissy",positive,None,None


In [45]:
good_test = test[0:219]
good_test = good_test.dropna()
good_test

,textID,text,selected_text,sentiment,predicted_text,predicted_sentiment
0,33537ffd4d,is excited to see Samantha&Denise again http:...,excited,positive,excited to see,positive
1,609bf831e2,apparently even Novell Moonlight doesn`t work ...,doesn`t work,negative,doesn`t work,negative
2,4db58698d0,__ Aww bless her. She`s one of my faves,bless her. She`s one of my faves,positive,Aww bless her. She`s one of my faves,positive
3,fb45b19ca3,hahah I`ll try,hahah I`ll try,neutral,hahah I`ll try,neutral
4,630285dbc7,bored,bored,neutral,bored,neutral
...,...,...,...,...,...,...
214,8b3a2058c5,my cousin is in jail for shoplifting and drug...,im upset,negative,"cousin is in jail for shoplifting and drugs, u...",negative
215,d240f7c6d8,Good Morning Hows u ?,Good Morning,positive,Good Morning Hows u ?,neutral
216,aea00d01f6,i`m not being mean,i`m not being mean,positive,i`m not being mean,neutral
217,d5c1749c3a,my teeth and head hurts,my teeth and head hurts,negative,teeth and head hurts,negative


In [46]:
for i in good_test.index:
    print(test['predicted_text'][i])

excited to see
doesn`t work
Aww bless her. She`s one of my faves
hahah I`ll try
bored
that was such a tease!  I cant wait til Sunday!!  but then after that, nothing until November   oh well, i can wait!
loveee
homeee! my feet are sore
Sittin at the hospital with isaac cause he hit his head
Watching the Killers on JR. They are awesome. Too late though. They just cut the programme mid song. Twas my favourite too.
Yesterday I didn`t do much because I was just lazy, but today I don`t feel so good.
Mother`s Day breakfast is over, getting back to work now. Back on Rails
going to EK nao... will meet bloggers there instead of Makati
Other people have their Glasgow ones, standing and seats, for all nights.
Translate XMind to your language.
everything counts  what are you drinking?
watching JONAS
Happy Mother`s Day!
G*morning! But I don`t care so much
I got the ticket dismissed
not allowed in the sun
I like Light out, Walls up & shatter`d the most! Great work, Germany must get to know you asap..

### Jaccard Similarity:- The tool we rely on to unveil the hidden connections and measure the overlap between sets of words, guiding us through the vast ocean of textual data.

In [47]:
def Jaccard_Similarity(sen1, sen2):
    words_sen1 = set(sen1.lower().split())
    words_sen2 = set(sen2.lower().split())
    
    intersection = words_sen1.intersection(words_sen2)
    
    union = words_sen1.union(words_sen2)
    
    return float(len(intersection))/len(union)

In [48]:
per_list = []
for j,i in enumerate(good_test.index):
    sen1 = test['selected_text'][i]
    sen2 = test['predicted_text'][i] 
    
    percentage = Jaccard_Similarity(sen1, sen2)
    print(percentage)
    per_list.append(percentage)

0.3333333333333333
1.0
0.875
1.0
1.0
0.9523809523809523
0.2
0.2
0.2727272727272727
0.3
0.3333333333333333
1.0
0.9166666666666666
1.0
1.0
1.0
1.0
0.3333333333333333
0.7142857142857143
1.0
0.3333333333333333
0.10526315789473684
0.0
1.0
1.0
1.0
1.0
0.17647058823529413
1.0
0.125
1.0
1.0
0.2
0.07692307692307693
0.25
1.0
0.3157894736842105
0.5
0.038461538461538464
1.0
0.08695652173913043
1.0
0.875
0.06666666666666667
0.1
1.0
0.6
0.125
0.2857142857142857
0.25
0.9333333333333333
0.18181818181818182
0.9166666666666666
0.25
1.0
0.125
1.0
0.0
0.6
1.0
0.2
0.5
0.8333333333333334
0.75
0.5
1.0
0.08333333333333333
1.0
0.3333333333333333
1.0
0.5
0.6666666666666666
0.2
0.5
0.125
0.3333333333333333
0.2
0.25
1.0
1.0
0.42857142857142855
0.9411764705882353
0.5333333333333333
0.9444444444444444
0.3333333333333333
1.0
1.0
0.1111111111111111
1.0
0.0
0.16666666666666666
0.6
1.0
0.3181818181818182
0.1
0.16
0.2
0.14285714285714285
1.0
0.32
0.125
0.5
0.5
0.1
1.0
0.09090909090909091
0.3333333333333333
0.5
0.75
0.0


### Accuracy (when we use 15 statements in prompt)

In [49]:
average = sum(per_list) / len(per_list)
print(average)

0.5447867869060422


## Test on 219 datasets

In [74]:
good_test= good_test[:219]
good_test

,textID,text,selected_text,sentiment,predicted_text,predicted_sentiment
0,33537ffd4d,is excited to see Samantha&Denise again http:...,excited,positive,excited to see,positive
1,609bf831e2,apparently even Novell Moonlight doesn`t work ...,doesn`t work,negative,doesn`t work,negative
2,4db58698d0,__ Aww bless her. She`s one of my faves,bless her. She`s one of my faves,positive,Aww bless her. She`s one of my faves,positive
3,fb45b19ca3,hahah I`ll try,hahah I`ll try,neutral,hahah I`ll try,neutral
4,630285dbc7,bored,bored,neutral,bored,neutral
...,...,...,...,...,...,...
214,8b3a2058c5,my cousin is in jail for shoplifting and drug...,im upset,negative,"cousin is in jail for shoplifting and drugs, u...",negative
215,d240f7c6d8,Good Morning Hows u ?,Good Morning,positive,Good Morning Hows u ?,neutral
216,aea00d01f6,i`m not being mean,i`m not being mean,positive,i`m not being mean,neutral
217,d5c1749c3a,my teeth and head hurts,my teeth and head hurts,negative,teeth and head hurts,negative


### Prompt Engineering

In [75]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [76]:
delimiter = "####"
system_message_1 = f"""
consider yourself as Human working on positive , negative or neural text classifier having great
deal of english language. Now given a text delimited with {delimiter} characters.
Highlight the parts of text which make it positive or negative or neutral.
Output should be in python dictionary format with keys being sentiment, part_of_text
and the values for sentiment and part_of_text should be single output string and should not have list of outputs for better understanding see examples bellow
where part_of_text contains only that words that have high similarity with the sentiments giving out different parts
of text which make it profane, vulgar, or offensive
Note that when sentiment is neutral, part_of_text simply contains user message and part_of_text contains only one index size

So, I need to train on these below examples
there will be 3 columns input_text,text_extracted,labels these columns will have values separated by comma 
and their descriptions are as follows
input_text = actual text data
text_extracted = text which actually provides emotions to the original text
labels = labels/sentiment assigned to extracted text

1. you are stupid , stupid ,negative
2. Sooo SAD I will miss you here in San Diego!!!, Sooo SAD, negative
3. my boss is bullying me..., bullying me, negative
4. what interview! leave me alone, leave me alone, negative
5. Sons of bitch why couldn`t they put them on the releases we already bought, Sons of bitch, negative
6. y do i even bother getting a new fone..i just **** break them anyway fones dont float especially when thrown in the pool, **** break th, negative 
7. The sun is attempting to increase her efforts. Alas the same cannot be said for me. I see a bath, a couch & a book in my immediate future, Alas,negative
8. Omg guys my internet has been down the whole day  It`s still not working  I`ll try fix 2m, Omg guys my internet has been down the whole day,negative
9. So sorry to hear your terrible news   Thinking of you all x, So sorry to hear your terrible news,negative
10. Shopped til i dropped....come bac sunshine i miss u, Shopped til i dropped....come bac sunshine i miss u,negative
11. there is absolutely nothing to eat in my house... epic fail, epic fail,negative
12. mom just woke me u[p and i am so mad i was dreaming about  shoes  she whants me to go to the river it is so stupid, o stupid,negative
13. So cold in here!  getting the chill , So cold in here!  getting the chill,negative
14. addin to that last comment, was spose to be going to see the jonas brothers 3D movie, but guess what i couldnt go aswell cos im sick, im sick,negative
15. I`m SO jealous- it`s pouring right now, jealous-,negative

16. A little happy for the wine jeje ok it`sm my free time so who cares jaja i love this day, A little happy fo, positive
17. lucky kid...i so wanna see loserville  pity im in oz...., lucky, positive
18. Happy Mothers day to all you Mums out there, Happy Mothers day to all you Mums out there, positive
19. Thank you, Afrin Nasal Spray! Also, I got a giant teacup tonight!, Thank you,, positive
20. Packing up for the trip to Ascari track, thank you Bacardi, thank you Bacardi, positive
21. Aha both same length, touche ,touche ,positive
22. I`m looking forward to going home tomorrow, but I really wish it was for a different reason., looking forward to,positive
23. _LOCA thank you babe  What are you doin this evening?, thank you ba,positive
24. Going home  but when i get home , BLACK BERRY ! WOO HOO  . . . and get to see presten ! <3, WOO HOO,positive
25. Working, Listening to music,Testing out a new singer/guitarist tonight, let`s hope for the best, hope,positive
26. its my birthday too  but its the 10th of may right now for me.. and its probably the 9th for you.. happy birthday anyway, . happy,positive
27. I am twittering, LIKE A BOSS. Thanks Savvv, Thanks Savvv,positive
28. Birthday Girl ?  blessed to live another year & celebrate with loved ones, blessed,positive
29. @__Jasper_Hale__ *Laughs* that`s because I like winning, like,positive
30. I love this!!  our body should tell us how much sleep it needs, and its always good to be mindful at mealtimes, s always good,positive

31. yeah, it`s a bit 80s silly at points but i love it, yeah, it`s a bit 80s silly at points but i love it, neutral
32. It's gonna be a super long day, It`s gonna be a super long day, neutral
33. I'm soooooooo hungry right now. DH is still not home, I`m soooooooo hungry right now. DH is still not home., neutral
34. Where`d the songs go on the site I want 'Do You' on this computer too, Where`d the songs go on the site I want 'Do You' on this computer too, neutral
35. fighting again, fighting again, neutral
36. omg i forgot about that! you lucky thing! hope you have fun!!  x, omg i forgot about that! you lucky thing! hope you have fun!!  x,neutral
37. . Really I look like a boy!!! I need someone to do my hair!, . Really I look like a boy!!! I need someone to do my hair!,neutral
38. _Cullen8 i hate the rain *laughs*. thanks *smiles* i miss her loads i couldnt stop crying before, _Cullen8 i hate the rain *laughs*. thanks *smiles* i miss her loads i couldnt stop crying before,neutral
39. Going to go kennel the dogs and then I am going to watch Goblet of Fire, Going to go kennel the dogs and then I am going to watch Goblet of Fire,neutral
40. later I went up to monroe and worked out with friends - TRX, ropes and KBs!!   30sec on/30 off..rope swings & c&ps, windmills, wve aquats, later I went up to monroe and worked out with friends - TRX, ropes and KBs!!   30sec on/30 off..rope swings & c&ps, windmills, wve aquats,neutral
41. _007 so I tried to send you a direct message and I can`t because you aren`t following me, 007 so I tried to send you a direct message and I can`t because you aren`t following me,neutral
42. Just watched the 'Final Break' final prison break episode. It was great. Farewell Prison Break. You will be dearly missed., Just watched the 'Final Break' final prison break episode. It was great. Farewell Prison Break. You will be dearly missed.,neutral
43. Im still in my my pjs  but I haveto get dressed cuz Im going to see Dancing on Ice live, Im still in my my pjs  but I haveto get dressed cuz Im going to see Dancing on Ice live,neutral
44. goodnight twitterville! really enjoyed pearl harbor. now off to sleep and cuddle with my hot man!  i will have good dreams tonight! XOXO, goodnight twitterville! really enjoyed pearl harbor. now off to sleep and cuddle with my hot man!  i will have good dreams tonight! XOXO,neutral
45. UGH... I am soooo drag assing today  but on a brighter note I got my G1 update & it is the biznesssss!, UGH... I am soooo drag assing today  but on a brighter note I got my G1 update & it is the biznesssss!,neutral
"""

In [79]:
from time import sleep
for j,i in enumerate(test.index):
    if i%3==0:
        sleep(60)
    user_message = good_test.iloc[i]['text']
    print(f'--->{user_message}\n')
    messages =  [  
    {'role':'system', 
     'content': system_message_1},    
    {'role':'user', 
     'content': f"{delimiter}{user_message}{delimiter}"},  
    ] 
    response = get_completion_from_messages(messages)
    try:
        response = json.loads(response.replace("'", '"'))
        print(j,response)
        print('\n')
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", j,e)
        continue
    selected_text = response.get('part_of_text')
    selected_sentiment = response.get('sentiment')
#     print(selected_text)
    good_test['predicted_text'][j] = selected_text
    good_test['predicted_sentiment'][j] = selected_sentiment
#     break

--->is excited to see Samantha&Denise again  http://plurk.com/p/rr5id

0 {'sentiment': 'positive', 'part_of_text': 'excited to see Samantha&Denise again'}


--->apparently even Novell Moonlight doesn`t work on my Jaunty



/tmp/ipykernel_8072/3424815842.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_test['predicted_text'][j] = selected_text
/tmp/ipykernel_8072/3424815842.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_test['predicted_sentiment'][j] = selected_sentiment


1 {'sentiment': 'negative', 'part_of_text': 'doesn`t work on my Jaunty'}


--->__ Aww bless her. She`s one of my faves

2 {'sentiment': 'positive', 'part_of_text': 'one of my faves'}


---> hahah I`ll try

Error decoding JSON: 3 Expecting value: line 1 column 1 (char 0)
---> bored

4 {'sentiment': 'neutral', 'part_of_text': 'bored'}


---> that was such a tease!  I cant wait til Sunday!!  but then after that, nothing until November   oh well, i can wait!

5 {'sentiment': 'neutral', 'part_of_text': 'that was such a tease!  I cant wait til Sunday!!  but then after that, nothing until November   oh well, i can wait!'}


--->  Joe and I loveee you.

6 {'sentiment': 'positive', 'part_of_text': 'loveee'}


--->homeee! my feet are sore

7 {'sentiment': 'neutral', 'part_of_text': 'homeee! my feet are sore'}


--->Sittin at the hospital with isaac cause he hit his head

8 {'sentiment': 'neutral', 'part_of_text': 'Sittin at the hospital with isaac cause he hit his head'}


--->Watching the Kille

52 {'sentiment': 'positive', 'part_of_text': 'Brilliant  Thanks!'}


---> no sleeping for me im at work   it will be summer break soon. Tell them to get there butts out of bed lol

53 {'sentiment': 'neutral', 'part_of_text': 'no sleeping for me im at work   it will be summer break soon. Tell them to get there butts out of bed lol'}


---> have a safe trip joshy poo.......you`ll knock them dead at your speech

Error decoding JSON: 54 Extra data: line 1 column 62 (char 61)
--->SB 239K.  Flop A Q 9.  4 players check. Turn 9. I check call 200 bet. River brick. I check call with trip 9`s with K kick. He shows A9.

55 {'sentiment': 'neutral', 'part_of_text': 'SB 239K.  Flop A Q 9.  4 players check. Turn 9. I check call 200 bet. River brick. I check call with trip 9`s with K kick. He shows A9.'}


--->Power Outage, door to freezer propped open, 3G ice cream make for slippy floor  M-er F-er!!

56 {'sentiment': 'negative', 'part_of_text': 'Power Outage, door to freezer propped open, 3G ice crea

101 {'sentiment': 'neutral', 'part_of_text': 'Ooo showing of your French skills!! lol Things good over here. Lovely weather, so should be outside  How`s u?'}


---> Thanks

Error decoding JSON: 102 Expecting value: line 1 column 1 (char 0)
--->I`m torn, do I get an extended warranty for my iPhone when mine runs out, or jailbreak? I`ve already got problems with battery & earpiece

103 {'sentiment': 'neutral', 'part_of_text': 'I`m torn, do I get an extended warranty for my iPhone when mine runs out, or jailbreak? I`ve already got problems with battery & earpiece'}


--->I am off to bed now....my family has a wonderful day planned for me today      Happy Mother`s Day

104 {'sentiment': 'positive', 'part_of_text': 'Happy Mother`s Day'}


--->no more brother for 2 weeks.....

105 {'sentiment': 'neutral', 'part_of_text': 'no more brother for 2 weeks.....'}


--->_Jo my mask is non-existent at the mo  Charis didn`t send me one & I haven`t been bothered to make one! I`m wearing boy clothes!

1

148 {'sentiment': 'neutral', 'part_of_text': 'Please visit http://vzerohost.com/info and sign up to alpha test a image hosting service!'}


--->wishing my days off were the same days that people wanted to party.

149 {'sentiment': 'negative', 'part_of_text': 'wishing my days off were the same days that people wanted to party.'}


--->back from lunch and it`s pouring rain! ugh  ohh well at least it gets me into a Death Cab For Cutie mood!

150 {'sentiment': 'negative', 'part_of_text': 'pouring rain! ugh'}


---> lol well thank you

151 {'sentiment': 'positive', 'part_of_text': 'thank you'}


--->UP was awesome! Everyone should go see it.  But the stupid theatre didn`t show the Pixar short film.

152 {'sentiment': 'positive', 'part_of_text': 'UP was awesome! Everyone should go see it.'}


--->You fooled me!

153 {'sentiment': 'neutral', 'part_of_text': 'You fooled me!'}


--->Starving. And Alyssa`s mom would bring her subway.

154 {'sentiment': 'negative', 'part_of_text': 'Starving'}


-

---> Happened to me the other day.  Was wearing a necklace that made a white spot on my chest.

201 {'sentiment': 'neutral', 'part_of_text': 'Happened to me the other day.  Was wearing a necklace that made a white spot on my chest.'}


--->mmm... hot chocolate, ugboots and topgear... life is good.

202 {'sentiment': 'positive', 'part_of_text': 'hot chocolate, life is good.'}


---> Your schmoo is moving? Are you sad?

203 {'sentiment': 'neutral', 'part_of_text': 'Your schmoo is moving? Are you sad?'}


---> OR... Something to do with all the Red Bull?  Hmmm. You think maybe?

204 {'sentiment': 'neutral', 'part_of_text': 'OR... Something to do with all the Red Bull?  Hmmm. You think maybe?'}


---> hahahaha awww uduhn lun me no mor???

205 {'sentiment': 'negative', 'part_of_text': 'no mor'}


--->Prolly going to Jamba Juice in thirty minutes , shop for awhile then hit the airport and see my daddddd.

206 {'sentiment': 'neutral', 'part_of_text': 'Prolly going to Jamba Juice in thirty min

IndexError: single positional indexer is out-of-bounds

In [81]:
good_test_1 = good_test[0:219]
good_test_1 = good_test_1.dropna()
good_test_1

,textID,text,selected_text,sentiment,predicted_text,predicted_sentiment
0,33537ffd4d,is excited to see Samantha&Denise again http:...,excited,positive,excited to see Samantha&Denise again,positive
1,609bf831e2,apparently even Novell Moonlight doesn`t work ...,doesn`t work,negative,doesn`t work on my Jaunty,negative
2,4db58698d0,__ Aww bless her. She`s one of my faves,bless her. She`s one of my faves,positive,one of my faves,positive
3,fb45b19ca3,hahah I`ll try,hahah I`ll try,neutral,hahah I`ll try,neutral
4,630285dbc7,bored,bored,neutral,bored,neutral
...,...,...,...,...,...,...
214,8b3a2058c5,my cousin is in jail for shoplifting and drug...,im upset,negative,"cousin is in jail for shoplifting and drugs, u...",negative
215,d240f7c6d8,Good Morning Hows u ?,Good Morning,positive,Good Morning Hows u ?,neutral
216,aea00d01f6,i`m not being mean,i`m not being mean,positive,i`m not being mean,neutral
217,d5c1749c3a,my teeth and head hurts,my teeth and head hurts,negative,my teeth and head hurts,negative


In [82]:
last_test = good_test_1.reset_index(drop=True)

In [83]:
last_test

,textID,text,selected_text,sentiment,predicted_text,predicted_sentiment
0,33537ffd4d,is excited to see Samantha&Denise again http:...,excited,positive,excited to see Samantha&Denise again,positive
1,609bf831e2,apparently even Novell Moonlight doesn`t work ...,doesn`t work,negative,doesn`t work on my Jaunty,negative
2,4db58698d0,__ Aww bless her. She`s one of my faves,bless her. She`s one of my faves,positive,one of my faves,positive
3,fb45b19ca3,hahah I`ll try,hahah I`ll try,neutral,hahah I`ll try,neutral
4,630285dbc7,bored,bored,neutral,bored,neutral
...,...,...,...,...,...,...
207,8b3a2058c5,my cousin is in jail for shoplifting and drug...,im upset,negative,"cousin is in jail for shoplifting and drugs, u...",negative
208,d240f7c6d8,Good Morning Hows u ?,Good Morning,positive,Good Morning Hows u ?,neutral
209,aea00d01f6,i`m not being mean,i`m not being mean,positive,i`m not being mean,neutral
210,d5c1749c3a,my teeth and head hurts,my teeth and head hurts,negative,my teeth and head hurts,negative


In [88]:
per_list = []
for j,i in enumerate(good_test.index):
    sen1 = last_test['selected_text'][i]
    sen2 = last_test['predicted_text'][i] 
    
    percentage = Jaccard_Similarity(sen1, sen2)
    print(percentage)
    per_list.append(percentage)

0.2
0.4
0.5714285714285714
1.0
1.0
0.9523809523809523
0.2
0.2
0.2727272727272727
0.0
0.5
1.0
1.0
1.0
1.0
1.0
1.0
0.3333333333333333
0.38461538461538464
0.8
0.3333333333333333
0.3333333333333333
0.0
1.0
1.0
1.0
1.0
0.17647058823529413
1.0
0.1111111111111111
0.3181818181818182
1.0
1.0
0.07692307692307693
0.25
1.0
0.3157894736842105
1.0
0.34615384615384615
0.875
0.0
1.0
0.875
0.42857142857142855
0.1
1.0
0.6
0.125
0.2857142857142857
0.25
0.9333333333333333
0.18181818181818182
0.9166666666666666
0.25
1.0
0.125
1.0
0.0
0.6
1.0
0.0
0.6666666666666666
0.8333333333333334
0.75
1.0
1.0
0.08333333333333333
1.0
0.3333333333333333
1.0
0.5
0.8333333333333334
0.2
0.5833333333333334
0.2857142857142857
0.3333333333333333
0.2
0.25
1.0
1.0
0.42857142857142855
0.7142857142857143
0.4117647058823529
0.3076923076923077
0.9444444444444444
0.3333333333333333
1.0
1.0
0.1111111111111111
1.0
0.0
0.21428571428571427
0.6
1.0
0.391304347826087
0.35714285714285715
0.16
0.6
0.3157894736842105
1.0
0.32
0.125
0.5
0.04166

KeyError: 212

### Accuracy (when we use 45 statements in prompt)

In [89]:
average = sum(per_list) / len(per_list)
print(average)

0.5602548798099986


### Save File 

In [90]:
# last_test.to_csv('/home/user/Desktop/Extra/56_score_data.csv', index=False)